# Bygga med Mistral-modeller

## Introduktion

Den här lektionen tar upp:
- En översikt av de olika Mistral-modellerna
- Förstå användningsområden och scenarier för varje modell
- Kodexempel som visar de unika egenskaperna hos varje modell


## Mistral-modellerna

I den här lektionen kommer vi att utforska tre olika Mistral-modeller:  
**Mistral Large**, **Mistral Small** och **Mistral Nemo**.

Var och en av dessa modeller finns gratis på Github Model marketplace. Koden i denna notebook kommer att använda dessa modeller för att köra koden. Här finns mer information om hur du använder Github Models för att [prototypa med AI-modeller](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 är för närvarande flaggskeppsmodellen från Mistral och är utformad för företagsanvändning.

Modellen är en uppgradering av den ursprungliga Mistral Large och erbjuder
-  Större kontextfönster – 128k jämfört med 32k
-  Bättre prestanda på matte- och kodningsuppgifter – 76,9 % genomsnittlig noggrannhet jämfört med 60,4 %
-  Förbättrad flerspråkig prestanda – språken inkluderar: engelska, franska, tyska, spanska, italienska, portugisiska, nederländska, ryska, kinesiska, japanska, koreanska, arabiska och hindi.

Med dessa funktioner utmärker sig Mistral Large på
- *Retrieval Augmented Generation (RAG)* – tack vare det större kontextfönstret
- *Function Calling* – den här modellen har inbyggt stöd för funktionsanrop vilket möjliggör integration med externa verktyg och API:er. Dessa anrop kan göras både parallellt eller efter varandra i sekventiell ordning.
- *Kodgenerering* – den här modellen är särskilt bra på att generera kod i Python, Java, TypeScript och C++.


### RAG-exempel med Mistral Large 2


I det här exemplet använder vi Mistral Large 2 för att köra ett RAG-mönster på ett textdokument. Frågan är skriven på koreanska och handlar om författarens aktiviteter före universitetet.

Den använder Cohere Embeddings Model för att skapa embeddingar av både textdokumentet och frågan. I det här exemplet används Python-paketet faiss som vektorlagring.

Prompten som skickas till Mistral-modellen innehåller både frågorna och de hämtade delarna som liknar frågan. Modellen ger sedan ett svar i naturligt språk.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small är en annan modell i Mistral-familjen under kategorin premier/enterprise. Som namnet antyder är denna modell en Small Language Model (SLM). Fördelarna med att använda Mistral Small är att den är: 
- Kostnadseffektiv jämfört med Mistral LLMs som Mistral Large och NeMo – 80% lägre pris
- Låg latens – snabbare svar jämfört med Mistrals LLMs
- Flexibel – kan användas i olika miljöer med färre begränsningar på nödvändiga resurser. 

Mistral Small passar utmärkt för: 
- Textbaserade uppgifter som sammanfattning, sentimentanalys och översättning. 
- Applikationer där frekventa förfrågningar görs tack vare dess kostnadseffektivitet 
- Låg-latens koduppgifter som granskning och kodförslag


## Jämförelse mellan Mistral Small och Mistral Large

För att visa skillnader i latens mellan Mistral Small och Large, kör cellerna nedan.

Du bör märka en skillnad i svarstider på mellan 3–5 sekunder. Notera också skillnader i svarens längd och stil på samma prompt.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Jämfört med de andra två modellerna som diskuteras i den här lektionen är Mistral NeMo den enda kostnadsfria modellen med en Apache2-licens.

Den ses som en uppgradering av den tidigare öppna LLM-modellen från Mistral, Mistral 7B.

Några andra egenskaper hos NeMo-modellen är:

- *Effektivare tokenisering:* Den här modellen använder Tekken-tokenizern istället för den mer vanligt förekommande tiktoken. Det ger bättre prestanda för fler språk och kod.

- *Finjustering:* Basmodellen är tillgänglig för finjustering. Det ger större flexibilitet för användningsområden där finjustering kan behövas.

- *Inbyggd funktionsanropning* – Precis som Mistral Large har den här modellen tränats på funktionsanrop. Det gör den unik som en av de första öppna modellerna med denna funktion.


## Mistral NeMo

Jämfört med de andra två modellerna som diskuteras i den här lektionen är Mistral NeMo den enda kostnadsfria modellen med en Apache2-licens.

Den ses som en uppgradering av den tidigare öppna LLM-modellen från Mistral, Mistral 7B.

Några andra egenskaper hos NeMo-modellen är:

- *Effektivare tokenisering:* Den här modellen använder Tekken-tokenizern istället för den mer vanligt förekommande tiktoken. Det ger bättre prestanda för fler språk och kod.

- *Finjustering:* Basmodellen finns tillgänglig för finjustering. Det ger större flexibilitet för användningsområden där finjustering kan behövas.

- *Inbyggd funktionsanropning* – Precis som Mistral Large har den här modellen tränats på funktionsanrop. Det gör den unik som en av de första öppna modellerna med den här förmågan.


### Jämföra tokenizers

I det här exemplet ska vi titta på hur Mistral NeMo hanterar tokenisering jämfört med Mistral Large.

Båda exemplen använder samma prompt, men du bör se att NeMo returnerar färre tokens än Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Lärandet slutar inte här, fortsätt resan

När du har slutfört denna lektion, ta en titt på vår [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) för att fortsätta utveckla dina kunskaper inom Generativ AI!



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiska översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk ska betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi tar inget ansvar för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
